# Time series using rioxarray and stackstac {.unnumbered}

This notebook demonstrates how to generate a timeseries via `rioxarray`, `pystac_client` and `stackstac`.

* Author: Aimee Barciauskas
* Lasted Updated Date: August 3, 2022

In [ ]:
import rioxarray
from pystac_client import Client

# Accessing the data

The data is in a protected bucket. Please request access by emailng aimee@developmentseed.org providing your affiliation, interest in or expected use of the dataset, and an AWS IAM role or user Amazon Resource Name (ARN). Once you have provided the role you will use to access the data, the team will add that role or user ARN to the permitted list of identities which can assume a role with read access to the data. The code in `create_rio_session.py` can then be used to assume this read-access role, identified in that code by the environment variable `AWS_ROLE`.

In [ ]:
# Skip this if running in AWS Sagemaker
%run -i 'create_rio_session.py'

# Declare your collection of interest

You can discover available collections the following ways:

* Use the `{STAC_API_URL}/collections` API endpoint (JSON response)
* Programmatically using `pystac` (see example in the `list-collections.ipynb` notebook
* In the STAC Browser: http://delta-staging-stac-browser.s3-website-us-east-1.amazonaws.com/

In [ ]:
STAC_API_URL = 'https://staging-stac.delta-backend.com/'
collection = 'no2-monthly'

## Use satsearch to discover items in the no2-monthly collection for China

In [ ]:
china_bbox = [
    73.675,
    18.198,
    135.026,
    53.459
]
datetime = "2000-01-01T00:00:00Z/2022-01-02T00:00:00Z"
client = Client.open(STAC_API_URL)

search = client.search(
    bbox=china_bbox,
    datetime=datetime,
    collections=[collection],
    limit=1000
)
items = list(search.items())

In [ ]:
len(items)

## Create an `xarray.DataArray` using `stackstac`

In [ ]:
stack = stackstac.stack(items)
stack = stack.assign_coords({"time": stack.start_datetime})
stack

## Clip the data to the bounding box for China

In [ ]:
# Subset to Bounding Box for China
subset = stack.rio.clip_box(
    minx=china_bbox[0],
    miny=china_bbox[1],
    maxx=china_bbox[2],
    maxy=china_bbox[3]
)
subset

## Select a band of data

There is just one band in this case, `cog_default`.

In [ ]:
# select the band default
data_band = subset.sel(band='cog_default')
data_band

In [ ]:
# Group data into months
dates = data_band.groupby('time')

In [ ]:
# Average over entire spatial bounding box for each month
means = dates.mean(dim=('x', 'y'))

In [ ]:
means.plot()